In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Shyam\\Desktop\\renal_tumor_classification_project'

In [10]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Shyamanth-2005/renal_tumor_classification.mlflow/"
os.environ["MLFLOW_TRACKING_USERNAME"]="Shyamanth-2005"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1d22cf43202806cbe2bce95333b79a30fa13b1cb"

In [11]:
import tensorflow as tf

In [12]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
  path_of_model: Path
  training_data: Path
  all_params: dict
  mlflow_uri: str
  params_image_size: list
  params_batch_size: int

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [22]:
class ConfigurationManager:
  def __init__(
    self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    
    create_directories([self.config.artifacts_root])
    
  def get_evaluation_config(self) -> EvaluationConfig:
    eval_config = EvaluationConfig(
      path_of_model="artifacts/training/model.h5",
      training_data="artifacts/data_ingestion/kidney-ct-scan-image",
      mlflow_uri="https://dagshub.com/Shyamanth-2005/renal_tumor_classification.mlflow",
      all_params=self.params,
      params_image_size=self.params.IMAGE_SIZE,
      params_batch_size=self.params.BATCH_SIZE
    )
    return eval_config

In [23]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        # Set or create experiment
        experiment_name = "Kidney Tumor Classification"
        try:
            experiment_id = mlflow.create_experiment(experiment_name)
        except:
            experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
        
        mlflow.set_experiment(experiment_name)
        
        with mlflow.start_run():
            # Log params individually, converting lists to strings
            mlflow.log_param("AUGMENTATION", self.config.all_params.AUGMENTATION)
            mlflow.log_param("IMAGE_SIZE", str(self.config.all_params.IMAGE_SIZE))
            mlflow.log_param("BATCH_SIZE", self.config.all_params.BATCH_SIZE)
            mlflow.log_param("INCLUDE_TOP", self.config.all_params.INCLUDE_TOP)
            mlflow.log_param("EPOCHS", self.config.all_params.EPOCHS)
            mlflow.log_param("CLASSES", self.config.all_params.CLASSES)
            mlflow.log_param("WEIGHTS", self.config.all_params.WEIGHTS)
            mlflow.log_param("LEARNING_RATE", self.config.all_params.LEARNING_RATE)
            
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [27]:
try:
  config = ConfigurationManager()
  eval_config = config.get_evaluation_config()
  evaluation = Evaluation(eval_config)
  evaluation.evaluation()
  evaluation.log_into_mlflow()
except Exception as e:
  raise e
  

[2025-12-31 12:47:49,632: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-31 12:47:49,643: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-31 12:47:49,653: INFO: common: created directory at: artifacts]


Found 139 images belonging to 2 classes.
9/9 [==============================] - 12s 1s/step - loss: 0.2077 - accuracy: 0.9568
[2025-12-31 12:48:03,634: INFO: common: json file saved at: scores.json]


2025/12/31 12:48:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-12-31 12:48:11,722: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Shyam\AppData\Local\Temp\tmpdgtsypp6\model\data\model\assets
[2025-12-31 12:48:12,423: INFO: builder_impl: Assets written to: C:\Users\Shyam\AppData\Local\Temp\tmpdgtsypp6\model\data\model\assets]


c:\Users\Shyam\anaconda3\envs\renal\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/12/31 12:49:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
